In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

wb = load_workbook('./crawler.xlsx')
ws = wb.active

for column in range(1, ws.max_column + 1):
    if ws.cell(1, column).value == 'Json_Src':
        column_src = column
    elif ws.cell(1, column).value == 'Picture':
        column_pic = column

print(f'总数量：{ws.max_row - 1}')

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(2, ws.max_row + 1):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global ws

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import json

        src = json.loads(ws.cell(a, column_src).value).get('0', '')

        # = = = = = = = = = = = = = = =

        if src == '':
            b = 0
            crawler_status = 'ok'
        else:
            import requests

            from crawler_configuration import get_header, get_proxy

            try:
                b = 0
                while True:

                    while True:
                        b += 1
                        try:
                            resp = requests.get(src,
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                timeout=(10, 10))
                            break
                        except:
                            continue

                    if resp.status_code == 200:
                        break

                # = = = = = = = = = = = = = = =

                from io import BytesIO
                from openpyxl.drawing.image import Image

                pic = Image(BytesIO(resp.content))

                # = = = = = = = = = = = = = = =

                ws.add_image(pic, f'{get_column_letter(column_pic)}{a}')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                pass

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {src}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
wb.save(f'''./pic-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''')
print('Done ~')

总数量：97
[状态：ok，尝试次数：1] - https://i.ebayimg.com/images/g/N54AAOSw~EVmqhyz/s-l500.jpg
[剩余数量：67] - [当前时间：13:49:31]

[状态：ok，尝试次数：1] - https://i.ebayimg.com/images/g/N~QAAOSwThhmcSvx/s-l500.jpg
[剩余数量：66] - [当前时间：13:49:31]

[状态：ok，尝试次数：1] - https://i.ebayimg.com/images/g/OeYAAOSwuhlmqhzR/s-l500.jpg
[剩余数量：65] - [当前时间：13:49:31]

[状态：ok，尝试次数：1] - https://i.ebayimg.com/images/g/H3sAAOSwXyNmqhzI/s-l500.jpg
[剩余数量：64] - [当前时间：13:49:31]

[状态：ok，尝试次数：1] - https://i.ebayimg.com/images/g/0rcAAOSwC5xmqhzD/s-l500.jpg
[剩余数量：63] - [当前时间：13:49:31]

[状态：ok，尝试次数：1] - https://i.ebayimg.com/images/g/7o0AAOSwqFdk1EmZ/s-l500.jpg
[剩余数量：62] - [当前时间：13:49:32]

[状态：ok，尝试次数：1] - https://i.ebayimg.com/images/g/PfkAAOSw1rVmcAkm/s-l500.jpg
[剩余数量：61] - [当前时间：13:49:32]

[状态：ok，尝试次数：1] - https://i.ebayimg.com/images/g/5w4AAOSw7GhmqhzL/s-l500.jpg
[剩余数量：60] - [当前时间：13:49:32]

[状态：ok，尝试次数：1] - https://i.ebayimg.com/images/g/1rEAAOSwwz1mqhzV/s-l500.jpg
[剩余数量：59] - [当前时间：13:49:32]

[状态：ok，尝试次数：1] - https://i.ebayimg.com/images/g/